In [0]:
!pip install praw

    100% |████████████████████████████████| 122kB 5.8MB/s 
    100% |████████████████████████████████| 204kB 23.8MB/s 


In [1]:
import praw
import pandas as pd

In [2]:
usrname = 'mpbot0'
cid = 'yUJmELsXRFcD6A'
csecret = 'VKquj58R_cCuv021SqH6DYkCUic'
psswd = 'abcd1234'

In [3]:
# create a Reddit instance
reddit = praw.Reddit(client_id=cid,
                     client_secret=csecret, password=psswd,
                     user_agent='aneesh', username=usrname)

In [6]:
# path = r"top200.csv"
path = r"C:\Users\anees\Documents\CS\MP\datasets\top200.csv"

In [7]:
slist = pd.read_csv(path, names=['subreddit'])
sublist_full = slist['subreddit']
print("No of subreddits: " + str(len(sublist_full)))
sublist = [x for x in sublist_full]
sublist = sublist[:10]

No of subreddits: 200


In [55]:
def getSubComments(comment, allComments, sub, verbose=False):
    c = comment.body
    if c not in ['[removed]','[deleted]'] and comment.author is not None:
        allComments.append([sub, comment.author.name, c, comment.score])
    if not hasattr(comment, "replies"):
        replies = comment.comments()
        if verbose: 
            print("fetching (" + str(len(allComments)) + " comments fetched total)")
    else:
        replies = comment.replies
    for child in replies:
        getSubComments(child, allComments, sub, verbose=verbose)

In [73]:
def getAll(submission, sub, verbose=False):
    comments = submission.comments
    submission.comments.replace_more(limit=0) # 0 for faster but less covering scrape
    commentsList = []
    for comment in comments:
        getSubComments(comment, commentsList,sub, verbose=verbose)
    return commentsList

In [75]:
full_comment = []
for sub in sublist:
    count = 0
    subreddit = reddit.subreddit(sub)
    try:
        next(subreddit.hot())
    except:
        print("quarantine encountered")
        subreddit.quaran.opt_in()
        
    subreddit = reddit.subreddit(sub)
    top = subreddit.top('month',limit=100)
    print("Current Subreddit: " + sub)
    comments = []
    step = 0
    for submission in top:
        if not submission.stickied:
            if count>=20000:
                break
            try:
                submission.comments.replace_more(limit=0)
                comment = getAll(submission,sub=sub)
                count+=len(comments)
                if count - step > 400:
                    print(count)
                    step = count
                comments.extend(comment)
            except praw.exceptions.PRAWException as e:
                pass
        if count>=20000:
            break
    full_comment.append(comments)

Current Subreddit: funny
490
1474
2952
4922


KeyboardInterrupt: 

In [51]:
comments

[]

In [0]:
full_comment = []
for sub in sublist:
    subreddit = reddit.subreddit(sub)
    try:
        next(subreddit.hot())
    except:
        print("quarantine encountered")
        subreddit.quaran.opt_in()
    subreddit = reddit.subreddit(sub)
    top = subreddit.top('month',limit=50)
    print("Current Subreddit: " + sub)
    count = 0
    comments = []
    for submission in top:
        if not submission.stickied:
            submission.comments.replace_more(limit=0)
            for comment in submission.comments.list():
                if count>=20000:
                    break
                try:
                    if count%500==0:
                        print(count)
                    c = comment.body
                    if c in ['[removed]','[deleted]']:
                        continue
                    score = comment.score
                    if comment.author is None:
                        continue
                    user = comment.author.name
                    comments.append([sub, user, c, score])
                    count+=1  
                except praw.exceptions.PRAWException as e:
                    pass
        if count>=20000:
            break
    full_comment.append(comments)

Current Subreddit: funny
0
500
1000
1500
Current Subreddit: AskReddit
0
500
1000
1500
Current Subreddit: todayilearned
0
500
1000
1500
Current Subreddit: worldnews
0
500
1000
1500
Current Subreddit: science
0
500
1000
1000
1000
1000
1000
1000
1500
Current Subreddit: pics
0
500
1000
1500
Current Subreddit: gaming
0
500
1000
1500
Current Subreddit: IAmA
0
500
1000
1500
Current Subreddit: videos
0
500
1000
1500
Current Subreddit: movies
0
500
1000
1500
Current Subreddit: aww
0
500
1000
1500
Current Subreddit: Music
0
500
1000
1500
Current Subreddit: blog
0
Current Subreddit: gifs
0
500
1000
1500
Current Subreddit: news
0
500
1000
1500
Current Subreddit: explainlikeimfive
0
500
1000
1500
Current Subreddit: askscience
0
0
0
0
500
1000
1500
Current Subreddit: EarthPorn
0
500
1000
1500
Current Subreddit: books
0
500
1000
1500
Current Subreddit: television
0
500
1000
1500
Current Subreddit: mildlyinteresting
0
500
1000
1500
Current Subreddit: LifeProTips
0
500
1000
1500
Current Subreddit: Show

In [0]:
comments = []
count = 0 
for sub in full_comment:
    for comment in sub:
        comments.append(comment)  
    count+=1

In [0]:
df = pd.DataFrame(columns=['subreddit', 'user', 'comment', 'score'])

In [0]:
df.comment = [x[2] for x in comments]
df.subreddit = [x[0] for x in comments]
df.user = [x[1] for x in comments]
df.score = [x[3] for x in comments]

In [0]:
df.head()

,subreddit,user,comment,score
0,funny,papa_mike2,At least check the mail while you’re down ther...,10749
1,funny,notagoodboye,He missed cracking his head open on that littl...,14585
2,funny,Rati0h,I’ve never seen shoes leave a mans body as vio...,7169
3,funny,whezel,“I didn’t fuckin do it on purpose” is the best...,16282
4,funny,MB8189,"The audio is perfect, “didn’t fucking do it on...",2380


In [0]:
df.to_csv("2000c_200sub.csv", index=False)